In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import string
import random
import glob

# Configuración de las credenciales de la API de Spotify
SPOTIPY_CLIENT_ID = '17c9343d292a460f80047204c41186b0'
SPOTIPY_CLIENT_SECRET = '67e53999fca343e9a0ea6a4afda0fb8c'


# Configuración del cliente de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                           client_secret=SPOTIPY_CLIENT_SECRET),
                     requests_timeout=10, retries=10)


# Descargar de Spotify un CSV con un rango de años a seleccionar

In [19]:

def generate_prefixes():
    """
    Genera una lista de 50 prefijos únicos para segmentar las consultas a la API.
    Utiliza combinaciones de letras y números para crear prefijos más específicos.
    """
    prefixes = []
    letters = string.ascii_lowercase  # 'a' a 'z'
    numbers = string.digits  # '0' a '9'

    # Agregar prefijos de una sola letra
    for letter in letters:
        prefixes.append(f"{letter}*")
        if len(prefixes) >= 50:
            break

    # Si aún no alcanzamos 50, agregar combinaciones de letra + número
    if len(prefixes) < 50:
        for letter in letters:
            for number in numbers:
                prefixes.append(f"{letter}{number}*")
                if len(prefixes) >= 50:
                    break
            if len(prefixes) >= 50:
                break

    return prefixes

def get_tracks_by_year(year, prefixes, processed_ids):
    """
    Obtiene todas las pistas lanzadas en un año específico utilizando prefijos de búsqueda.
    Filtra las pistas ya procesadas para evitar duplicados.
    """
    all_tracks = []
    
    for prefix in prefixes:
        print(f"Fetching tracks for query '{prefix}' in year {year}...")
        try:
            results = sp.search(q=f'{prefix} year:{year}', type='track', limit=50)
        except Exception as e:
            print(f"Error en la consulta '{prefix}' para el año {year}: {e}")
            continue

        tracks = results['tracks']['items']
        total = min(results['tracks']['total'], 1000)  # Límite de 1000 resultados por consulta
        fetched = len(tracks)
        offset = 50

        # Filtrar pistas ya procesadas
        new_tracks = [track for track in tracks if track['id'] not in processed_ids]
        all_tracks.extend(new_tracks)
        processed_ids.update(track['id'] for track in new_tracks if track.get('id'))

        # Manejar paginación
        while fetched < total:
            try:
                time.sleep(random.uniform(0.1, 0.3))  # Retraso aleatorio para respetar límites de tasa
                results = sp.search(q=f'{prefix} year:{year}', type='track', limit=50, offset=offset)
                tracks = results['tracks']['items']
                new_tracks = [track for track in tracks if track['id'] not in processed_ids]
                all_tracks.extend(new_tracks)
                processed_ids.update(track['id'] for track in new_tracks if track.get('id'))
                fetched += len(new_tracks)
                offset += 50

                if len(new_tracks) == 0:
                    break
            except Exception as e:
                print(f"Error en la paginación de '{prefix}' para el año {year}: {e}")
                break

        time.sleep(random.uniform(0.1, 0.3))  # Retraso aleatorio entre consultas

    return all_tracks

def extract_track_info(tracks, artist_info_dict, audio_features_dict):
    """
    Extrae la información relevante de una lista de pistas.
    """
    track_info_list = []

    # Recopilar IDs de pistas y artistas
    track_ids = [track['id'] for track in tracks if track.get('id')]
    artist_ids = [track['artists'][0]['id'] for track in tracks if track['artists']]

    # Solicitudes en lote para características de audio
    audio_features_list = []
    for i in range(0, len(track_ids), 100):
        batch_ids = track_ids[i:i+100]
        try:
            audio_features = sp.audio_features(batch_ids)
            audio_features_list.extend(audio_features)
        except Exception as e:
            print(f"Error al obtener características de audio para el lote {i//100 +1}: {e}")
        time.sleep(random.uniform(0.1, 0.3))  # Retraso aleatorio

    # Solicitudes en lote para información de artistas
    unique_artist_ids = list(set(artist_ids))
    for i in range(0, len(unique_artist_ids), 50):  # Máximo 50 IDs por solicitud
        batch_ids = unique_artist_ids[i:i+50]
        try:
            artists = sp.artists(batch_ids)['artists']
            for artist in artists:
                artist_info_dict[artist['id']] = artist.get('genres', [])
        except Exception as e:
            print(f"Error al obtener información de artistas para el lote {i//50 +1}: {e}")
        time.sleep(random.uniform(0.1, 0.3))  # Retraso aleatorio

    # Mapear IDs de pistas a características de audio
    audio_features_dict.update({af['id']: af for af in audio_features_list if af})

    for track in tracks:
        track_id = track.get('id')
        if not track_id:
            continue

        audio_features = audio_features_dict.get(track_id, {})
        artist_id = track['artists'][0]['id'] if track['artists'] else None
        genres = artist_info_dict.get(artist_id, []) if artist_id else []

        track_info = {
            'id': track_id,
            'name': track.get('name'),
            'artists': ', '.join(artist['name'] for artist in track.get('artists', [])),
            'duration_ms': track.get('duration_ms'),
            'release_date': track['album'].get('release_date') if track.get('album') else None,
            'year': track['album'].get('release_date', '')[:4] if track.get('album') else None,
            'acousticness': audio_features.get('acousticness'),
            'danceability': audio_features.get('danceability'),
            'energy': audio_features.get('energy'),
            'instrumentalness': audio_features.get('instrumentalness'),
            'liveness': audio_features.get('liveness'),
            'loudness': audio_features.get('loudness'),
            'speechiness': audio_features.get('speechiness'),
            'tempo': audio_features.get('tempo'),
            'valence': audio_features.get('valence'),
            'mode': audio_features.get('mode'),
            'key': audio_features.get('key'),
            'popularity': track.get('popularity'),
            'explicit': track.get('explicit'),
            'genre': ', '.join(genres)
        }

        track_info_list.append(track_info)

    return track_info_list

def create_dataframe(year, prefixes, processed_ids, artist_info_dict, audio_features_dict):
    """
    Crea un DataFrame con la información de las pistas para un año específico.
    """
    all_tracks = get_tracks_by_year(year, prefixes, processed_ids)
    print(f"Total unique tracks fetched for year {year}: {len(all_tracks)}")

    # Extraer información de las pistas
    track_info_list = extract_track_info(all_tracks, artist_info_dict, audio_features_dict)

    df = pd.DataFrame(track_info_list)
    return df


def combine_csv_files(start_year, end_year, output_filename):
    """
    Combina múltiples archivos CSV en un único DataFrame y elimina duplicados basados en 'id'.
    Guarda el DataFrame combinado en un archivo CSV.
    """
    df_list = []
    processed_ids = set()
    artist_info_dict = {}
    audio_features_dict = {}
    prefixes = generate_prefixes()

    for year_to_test in range(start_year, end_year + 1):
        try:
            # Crear DataFrame para el año especificado
            df = create_dataframe(year_to_test, prefixes, processed_ids, artist_info_dict, audio_features_dict)
            df_list.append(df)

            # Guardar el DataFrame individual en un archivo CSV
            df.to_csv(f'spotify_tracks_{year_to_test}.csv', index=False)
            print(f"Dataset guardado en spotify_tracks_{year_to_test}.csv")
        except Exception as e:
            print(f"Ocurrió un error en el año {year_to_test}: {e}")

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"Total tracks antes de eliminar duplicados: {len(combined_df)}")

        # Eliminar duplicados basados en la columna 'id'
        combined_df.drop_duplicates(subset=['id'], inplace=True)
        print(f"Total tracks después de eliminar duplicados: {len(combined_df)}")

        # Guardar el DataFrame combinado y limpio en un archivo CSV
        combined_df.to_csv(output_filename, index=False)
        print(f"Dataset combinado y limpio guardado en {output_filename}")
    else:
        print("No se encontraron DataFrames para combinar.")


if __name__ == "__main__":
    # Definir el rango de años que deseas procesar
    start_year = 2024
    end_year = 2024

    # Nombre del archivo CSV combinado final
    output_filename = 'spotify_tracks_combinado_unico.csv'

    # Ejecutar la combinación y limpieza de datos
    combine_csv_files(start_year, end_year, output_filename)


Fetching tracks for query 'a*' in year 2024...
Fetching tracks for query 'b*' in year 2024...
Fetching tracks for query 'c*' in year 2024...
Fetching tracks for query 'd*' in year 2024...
Fetching tracks for query 'e*' in year 2024...
Fetching tracks for query 'f*' in year 2024...
Fetching tracks for query 'g*' in year 2024...
Fetching tracks for query 'h*' in year 2024...
Fetching tracks for query 'i*' in year 2024...
Fetching tracks for query 'j*' in year 2024...
Fetching tracks for query 'k*' in year 2024...
Fetching tracks for query 'l*' in year 2024...
Fetching tracks for query 'm*' in year 2024...
Fetching tracks for query 'n*' in year 2024...
Fetching tracks for query 'o*' in year 2024...
Fetching tracks for query 'p*' in year 2024...
Fetching tracks for query 'q*' in year 2024...
Fetching tracks for query 'r*' in year 2024...
Fetching tracks for query 's*' in year 2024...
Fetching tracks for query 't*' in year 2024...
Fetching tracks for query 'u*' in year 2024...
Fetching trac

# Crear dos archivos CSV
Al principio tenía errores y me había quedado un archivo de más de 100 Mb, por eso había decidido partirlo en dos.

In [10]:
import pandas as pd
import glob

# Definir la ruta donde están guardados los archivos CSV
ruta_archivos = 'spotify_tracks_*.csv'  # El asterisco actúa como comodín para los años

# Utilizar glob para obtener una lista de todos los archivos que coincidan con el patrón
archivos_csv = glob.glob(ruta_archivos)

# Crear listas separadas para los archivos de 1920-1972 y 1973-2024
archivos_1920_1972 = [archivo for archivo in archivos_csv if any(año in archivo for año in map(str, range(1920, 1973)))]
archivos_1973_2024 = [archivo for archivo in archivos_csv if any(año in archivo for año in map(str, range(1973, 2025)))]

# Función para combinar archivos CSV
def combinar_csv(archivos, archivo_salida):
    dataframes = []
    
    for archivo in archivos:
        print(f"Leyendo archivo: {archivo}")
        df = pd.read_csv(archivo)
        dataframes.append(df)
    
    # Concatenar todos los DataFrames
    df_combinado = pd.concat(dataframes, ignore_index=True)
    
    # Guardar el DataFrame combinado en un archivo CSV
    df_combinado.to_csv(archivo_salida, index=False)
    print(f"Archivo combinado guardado como {archivo_salida}")

# Combinar y guardar los archivos de 1920-1972
combinar_csv(archivos_1920_1972, 'spotify_tracks_1920_1972.csv')

# Combinar y guardar los archivos de 1973-2024
combinar_csv(archivos_1973_2024, 'spotify_tracks_1973_2024.csv')


Leyendo archivo: spotify_tracks_1920.csv
Leyendo archivo: spotify_tracks_1921.csv
Leyendo archivo: spotify_tracks_1922.csv
Leyendo archivo: spotify_tracks_1923.csv
Leyendo archivo: spotify_tracks_1924.csv
Leyendo archivo: spotify_tracks_1925.csv
Leyendo archivo: spotify_tracks_1926.csv
Leyendo archivo: spotify_tracks_1927.csv
Leyendo archivo: spotify_tracks_1928.csv
Leyendo archivo: spotify_tracks_1929.csv
Leyendo archivo: spotify_tracks_1930.csv
Leyendo archivo: spotify_tracks_1931.csv
Leyendo archivo: spotify_tracks_1932.csv
Leyendo archivo: spotify_tracks_1933.csv
Leyendo archivo: spotify_tracks_1934.csv
Leyendo archivo: spotify_tracks_1935.csv
Leyendo archivo: spotify_tracks_1936.csv
Leyendo archivo: spotify_tracks_1937.csv
Leyendo archivo: spotify_tracks_1938.csv
Leyendo archivo: spotify_tracks_1939.csv
Leyendo archivo: spotify_tracks_1940.csv
Leyendo archivo: spotify_tracks_1941.csv
Leyendo archivo: spotify_tracks_1942.csv
Leyendo archivo: spotify_tracks_1943.csv
Leyendo archivo:

In [5]:
df_combinado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506524 entries, 0 to 506523
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                506524 non-null  object 
 1   name              506524 non-null  object 
 2   artists           506522 non-null  object 
 3   duration_ms       506524 non-null  int64  
 4   release_date      506524 non-null  object 
 5   year              506524 non-null  int64  
 6   acousticness      501568 non-null  float64
 7   danceability      501568 non-null  float64
 8   energy            501568 non-null  float64
 9   instrumentalness  501568 non-null  float64
 10  liveness          501568 non-null  float64
 11  loudness          501568 non-null  float64
 12  speechiness       501568 non-null  float64
 13  tempo             501568 non-null  float64
 14  valence           501568 non-null  float64
 15  mode              501568 non-null  float64
 16  key               50

In [7]:
df_combinado.sample(5)

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,genre
459037,1vvnYpYEMVB4aq9I6tHIEB,4 Your Eyez Only,J. Cole,530253,2016-12-09,2016,0.367,0.701,0.4850,0.000063,0.0852,-10.305,0.3640,78.863,0.496,0.0,11.0,65,True,"conscious hip hop, hip hop, north carolina hip..."
82561,1yegDiVa8d94H07ga1SLOt,Days Of Wine And Roses,Wes Montgomery,226200,1963,1963,0.991,0.470,0.0296,0.948000,0.0976,-25.411,0.0352,89.408,0.184,0.0,7.0,51,False,"hard bop, jazz, jazz guitar"
168061,4f745BdlZtDEmJ0W8ktHj7,La Carta Que Te Mandé,Carlos Y Jose,142013,1984-01-01,1984,0.730,0.760,0.5630,0.000091,0.0869,-4.992,0.0315,110.173,0.962,1.0,6.0,23,False,"musica neoleonesa, musica sonorense, norteno"
343157,4cuLQvvOWdFYlR84f4ApCJ,Come Together,"Trapt, An0maly",233582,2021-03-03,2021,0.105,0.706,0.6310,0.000413,0.1090,-7.648,0.0470,136.066,0.359,0.0,2.0,30,False,"alternative metal, nu metal, post-grunge"
98751,4TIzgFEl8Ou59Eca4ZqpSU,Lo Que Sea,Kako y Su Orquesta,268053,1967-11-18,1967,0.422,0.772,0.4910,0.023600,0.0615,-10.335,0.0383,125.772,0.929,0.0,2.0,5,False,"latin jazz, musica tradicional cubana, salsa i..."


# Crear un archivo CSV unificado

In [20]:
import pandas as pd
import glob

# Definir la ruta donde están guardados los archivos CSV
ruta_archivos = 'spotify_tracks_*.csv'  # El asterisco actúa como comodín para los años

# Utilizar glob para obtener una lista de todos los archivos que coincidan con el patrón
archivos_csv = glob.glob(ruta_archivos)

# Crear una lista vacía para almacenar los DataFrames
dataframes = []

# Iterar sobre la lista de archivos CSV y agregarlos a la lista de DataFrames
for archivo in archivos_csv:
    print(f"Leyendo archivo: {archivo}")
    df = pd.read_csv(archivo)
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame combinado en un archivo CSV
archivo_salida = 'spotify_tracks_combinado_1920_2024.csv'
df_combinado.to_csv(archivo_salida, index=False)

print(f"Archivo combinado guardado como {archivo_salida}")


Leyendo archivo: spotify_tracks_1920.csv
Leyendo archivo: spotify_tracks_1921.csv
Leyendo archivo: spotify_tracks_1922.csv
Leyendo archivo: spotify_tracks_1923.csv
Leyendo archivo: spotify_tracks_1924.csv
Leyendo archivo: spotify_tracks_1925.csv
Leyendo archivo: spotify_tracks_1926.csv
Leyendo archivo: spotify_tracks_1927.csv
Leyendo archivo: spotify_tracks_1928.csv
Leyendo archivo: spotify_tracks_1929.csv
Leyendo archivo: spotify_tracks_1930.csv
Leyendo archivo: spotify_tracks_1931.csv
Leyendo archivo: spotify_tracks_1932.csv
Leyendo archivo: spotify_tracks_1933.csv
Leyendo archivo: spotify_tracks_1934.csv
Leyendo archivo: spotify_tracks_1935.csv
Leyendo archivo: spotify_tracks_1936.csv
Leyendo archivo: spotify_tracks_1937.csv
Leyendo archivo: spotify_tracks_1938.csv
Leyendo archivo: spotify_tracks_1939.csv
Leyendo archivo: spotify_tracks_1940.csv
Leyendo archivo: spotify_tracks_1941.csv
Leyendo archivo: spotify_tracks_1942.csv
Leyendo archivo: spotify_tracks_1943.csv
Leyendo archivo: